In [1]:
# imports
import os

import pandas as pd
from sklearn.metrics import classification_report, f1_score

In [2]:
# configs
PROJECT_PATH = "/home/alex/paper-2025-anonymous-submission"

In [3]:
# load data
df_gigachat_lite = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_detection_gigachat_lite_predictions_extended.json"
    ),
    orient="index"
)
df_gigachat_max = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_detection_gigachat_max_predictions_extended.json"
    ),
    orient="index"
)
df_yandexgpt = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_detection_yandex_gpt4_predictions_extended.json"
    ),
    orient="index"
)
df_mistral = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_detection_mistral_nemo_predictions_extended.json"
    ),
    orient="index"
)
df_gpt4 = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_detection_gpt_4o_predictions_extended.json"
    ),
    orient="index"
)
dataset = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_data/dataset.json"
    ),
    orient="index"
)

/home/alex/aspirantura_hse/general_venv/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/home/alex/aspirantura_hse/general_venv/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [4]:
# explore GigaChat-Lite

def clear_gigachar_lite_predictions(text_pred):
    text_pred = text_pred.replace("<|message_sep|>", "")
    text_pred = text_pred.replace(".", "")
    text_pred = str.lower(text_pred)
    if "заголовок содержит игру слов" in text_pred:
        text_pred = "да"
    if "заголовок содержит вопрос, который может быть воспринят как игра" in text_pred:
        text_pred = "да"
    return text_pred

def binary_convert(text_pred):
    if "да" in text_pred:
        return True
    else:
        return False

df_gigachat_lite.head(2)

,user_prompt,system_prompt,gigachat_lite_preds
0,Заголовок новости: Комфортная среда. Cодержани...,Присутствует ли в заголовке новости игра слов?...,Да<|message_sep|>
1,Заголовок новости: Свинина стушевалась. Cодерж...,Присутствует ли в заголовке новости игра слов?...,Нет<|message_sep|>


In [5]:
df_gigachat_lite["gigachat_lite_preds"].value_counts()

Да<|message_sep|>                                                                                                                                                                                                                                1688
нет<|message_sep|>                                                                                                                                                                                                                                509
Не знаю.<|message_sep|>                                                                                                                                                                                                                           159
не знаю<|message_sep|>                                                                                                                                                                                                                            119
Нет<|message_sep

In [6]:
df_gigachat_lite["gigachat_lite_preds"] = df_gigachat_lite["gigachat_lite_preds"].apply(clear_gigachar_lite_predictions)

In [7]:
df_gigachat_lite["gigachat_lite_preds"].value_counts()

да         1695
нет         527
не знаю     278
Name: gigachat_lite_preds, dtype: int64

In [8]:
df_gigachat_lite["gigachat_lite_preds"] = df_gigachat_lite["gigachat_lite_preds"].apply(binary_convert)

In [9]:
df_gigachat_lite["gigachat_lite_preds"].value_counts()

True     1695
False     805
Name: gigachat_lite_preds, dtype: int64

In [10]:
print(classification_report(dataset["is_word_play"], df_gigachat_lite["gigachat_lite_preds"]))

              precision    recall  f1-score   support

       False       0.57      0.36      0.44      1259
        True       0.53      0.72      0.61      1241

    accuracy                           0.54      2500
   macro avg       0.55      0.54      0.52      2500
weighted avg       0.55      0.54      0.52      2500



In [11]:
# explore GigaChat-Max

df_gigachat_max.head(2)

,user_prompt,system_prompt,gigachat_max_pred
0,Заголовок новости: Комфортная среда. Cодержани...,Присутствует ли в заголовке новости игра слов?...,нет
1,Заголовок новости: Свинина стушевалась. Cодерж...,Присутствует ли в заголовке новости игра слов?...,да


In [12]:
df_gigachat_max["gigachat_max_pred"].value_counts()

да                                                                                                                         1062
нет                                                                                                                        1050
как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.     140
что-то в вашем вопросе меня смущает. может, поговорим на другую тему?                                                       127
не люблю менять тему разговора, но вот сейчас тот самый случай.                                                             119
не знаю                                                                                                                       2
Name: gigachat_max_pred, dtype: int64

In [13]:
def clear_gigachat_max_predictions(text_pred):
    if text_pred.strip() == "нет":
        return "нет"
    elif text_pred.strip() == "да":
        return "да"
    elif text_pred.strip() == "не знаю":
        return "не знаю"
    else:
        return "нет"

In [14]:
df_gigachat_max["gigachat_max_pred"] = df_gigachat_max["gigachat_max_pred"].apply(clear_gigachat_max_predictions)

In [15]:
df_gigachat_max["gigachat_max_pred"].value_counts()

нет        1436
да         1062
не знаю       2
Name: gigachat_max_pred, dtype: int64

In [16]:
df_gigachat_max["gigachat_max_pred"] = df_gigachat_max["gigachat_max_pred"].apply(binary_convert)

In [17]:
print(classification_report(dataset["is_word_play"], df_gigachat_max["gigachat_max_pred"]))

              precision    recall  f1-score   support

       False       0.64      0.73      0.68      1259
        True       0.68      0.59      0.63      1241

    accuracy                           0.66      2500
   macro avg       0.66      0.66      0.66      2500
weighted avg       0.66      0.66      0.66      2500



In [18]:
# yandex gpt4

df_yandexgpt.head(2)

,user_prompt,system_prompt,yagpt_pred
0,Заголовок новости: Комфортная среда. Cодержани...,Присутствует ли в заголовке новости игра слов?...,нет
1,Заголовок новости: Свинина стушевалась. Cодерж...,Присутствует ли в заголовке новости игра слов?...,да


In [19]:
df_yandexgpt["yagpt_pred"].value_counts()

нет                                                                                                           866
в интернете есть много сайтов с информацией на эту тему. [посмотрите, что нашлось в поиске](https://ya.ru)    629
не знаю                                                                                                       620
да                                                                                                            385
Name: yagpt_pred, dtype: int64

In [20]:
def clear_yandex_predictions(text_pred):
    if text_pred.strip() == "нет":
        return "нет"
    elif text_pred.strip() == "да":
        return "да"
    elif text_pred.strip() == "не знаю":
        return "не знаю"
    else:
        return "нет"

In [21]:
df_yandexgpt["yagpt_pred"] = df_yandexgpt["yagpt_pred"].apply(clear_yandex_predictions)

In [22]:
df_yandexgpt["yagpt_pred"].value_counts()

нет        1495
не знаю     620
да          385
Name: yagpt_pred, dtype: int64

In [23]:
df_yandexgpt["yagpt_pred"] = df_yandexgpt["yagpt_pred"].apply(binary_convert)

In [24]:
print(classification_report(dataset["is_word_play"], df_yandexgpt["yagpt_pred"]))

              precision    recall  f1-score   support

       False       0.55      0.93      0.69      1259
        True       0.76      0.24      0.36      1241

    accuracy                           0.58      2500
   macro avg       0.66      0.58      0.53      2500
weighted avg       0.66      0.58      0.53      2500



In [25]:
# mistral
df_mistral.head(2)

,user_prompt,system_prompt,mistral_nemo_preds
0,Заголовок новости: Комфортная среда. Cодержани...,Присутствует ли в заголовке новости игра слов?...,"[{'generated_text': [{'role': 'system', 'conte..."
1,Заголовок новости: Свинина стушевалась. Cодерж...,Присутствует ли в заголовке новости игра слов?...,"[{'generated_text': [{'role': 'system', 'conte..."


In [26]:
def clear_mistral_prediction(text_pred):
    
    text = text_pred[0]["generated_text"][-1]["content"]

    return text

In [27]:
df_mistral["mistral_nemo_preds"] = df_mistral["mistral_nemo_preds"].apply(clear_mistral_prediction)

In [28]:
df_mistral["mistral_nemo_preds"].value_counts()

Нет.    2500
Name: mistral_nemo_preds, dtype: int64

In [29]:
# GPT4-o

df_gpt4.head()

,json_response,task_id,gpt4o_yes_no
0,{'id': 'batch_req_679779eee74c8190b840d4b92f50...,0,нет
1,{'id': 'batch_req_679779eefd548190924faf63afa7...,1,да
2,{'id': 'batch_req_679779ef155881909d67c0621d40...,2,да
3,{'id': 'batch_req_679779ef2c148190a374f73b7b29...,3,да
4,{'id': 'batch_req_679779ef41888190a3b9b01c7b9e...,4,да


In [30]:
df_gpt4["gpt4o_yes_no"].value_counts()

да         1665
нет         813
не знаю      13
Да.           5
Нет.          4
Name: gpt4o_yes_no, dtype: int64

In [31]:
def clear_gpt4o_prediction(text_pred):
    
    text = str.lower(text_pred)
    text = text.replace(".", "")

    return text

In [32]:
df_gpt4["gpt4o_yes_no"] = df_gpt4["gpt4o_yes_no"].apply(clear_gpt4o_prediction)

In [33]:
df_gpt4["gpt4o_yes_no"].value_counts()

да         1670
нет         817
не знаю      13
Name: gpt4o_yes_no, dtype: int64

In [34]:
df_gpt4["gpt4o_yes_no"] = df_gpt4["gpt4o_yes_no"].apply(binary_convert)

In [35]:
print(classification_report(dataset["is_word_play"], df_gpt4["gpt4o_yes_no"]))

              precision    recall  f1-score   support

       False       0.82      0.54      0.65      1259
        True       0.65      0.88      0.75      1241

    accuracy                           0.71      2500
   macro avg       0.74      0.71      0.70      2500
weighted avg       0.74      0.71      0.70      2500



In [36]:
dataset["gigachat_lite_extended"] = df_gigachat_lite["gigachat_lite_preds"]
dataset["gigachat_max_extended"] = df_gigachat_max["gigachat_max_pred"]
dataset["yagpt4_extended"] = df_yandexgpt["yagpt_pred"]
dataset["mistral_extended"] = [False for _ in range(len(dataset))]
dataset["gpt4o_extended"] = df_gpt4["gpt4o_yes_no"]

In [37]:
dataset.to_csv(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_predictions/classification_extended.csv"
    )
)